In [3]:
import os
import sys
import gmsh
import numpy as np
import import_ipynb
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.tri import Triangulation
from scipy.constants import mu_0, epsilon_0, speed_of_light
from scipy.special import jvp, hankel2, h2vp, jv

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))
from fem_pos_processing import graph_results

# Constants

In [4]:
OMEGA = 2 * np.pi * 3E8
K0 = OMEGA * np.sqrt(mu_0 * epsilon_0)
WAVELENGTH = 2 * np.pi / K0

RADII = {'a': WAVELENGTH/2, 
            'R1': WAVELENGTH * 0.75, 'R2': WAVELENGTH, 
            'R3': WAVELENGTH * 1.25, 'R4': WAVELENGTH * 1.5}

# `create_domain()`

In [5]:
def create_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, h, R='R1', auto_save=True, view_mesh=False):
    # Define os parâmetros de entrada
    type, order = FINITE_ELEMENT

    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("circular_pec")
    factory = gmsh.model.occ

    # Criar os círculos com `gmsh.model.occ.addCircle`
    gamma_s = factory.addCircle(0, 0, 0, RADII['a'])
    gamma_a = factory.addCircle(0, 0, 0, RADII[R])

    # Criar loops das curvas
    scatterer = factory.addCurveLoop([gamma_s])
    artificial_domain = factory.addCurveLoop([gamma_a])

    # Criar superfícies para os dielétricos
    free_space = factory.addPlaneSurface([artificial_domain, scatterer])

    # Sincronizar geometria
    factory.synchronize()

    # Adicionar grupos físicos para Dim=1
    for i, tag in enumerate([gamma_s, gamma_a]):
        gmsh.model.addPhysicalGroup(1, [tag], tag=BOUNDARY[i]['tag'], name=BOUNDARY[i]['name'])

    # Adicionar grupos físicos para Dim=2
    gmsh.model.addPhysicalGroup(2, [free_space], tag=MATERIAL[0]['tag'], name=MATERIAL[0]['name'])

    # Refinar a malha na borda do espalhador
    h_refined = h / 2  # Tamanho menor para o espalhador
    gmsh.model.mesh.field.add("Distance", 1)
    gmsh.model.mesh.field.setNumbers(1, "CurvesList", [gamma_s])
    gmsh.model.mesh.field.setNumber(1, "Sampling", 100)

    # Definir um campo de tamanho para refinamento local
    gmsh.model.mesh.field.add("Threshold", 2)
    gmsh.model.mesh.field.setNumber(2, "InField", 1)
    gmsh.model.mesh.field.setNumber(2, "SizeMin", h_refined)
    gmsh.model.mesh.field.setNumber(2, "SizeMax", h)
    gmsh.model.mesh.field.setNumber(2, "DistMin", RADII['a'] / 10)
    gmsh.model.mesh.field.setNumber(2, "DistMax", RADII['a'])

    # Ativar o campo de malha
    gmsh.model.mesh.field.setAsBackgroundMesh(2)

    # Gerar malha 2D
    gmsh.option.setNumber("Mesh.SaveAll", 1)

    # Definir ordem dos elementos
    # gmsh.option.setNumber("Mesh.MeshSizeMax", h)
    # gmsh.option.setNumber("Mesh.MeshSizeMin", h)
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(order)

    # Visualizar a malha no ambiente Gmsh (opcional)
    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/circular_pec_domain_{type}{order}.msh"
        gmsh.write(file_path)
        gmsh.finalize()

# `apply_physics()`

In [6]:
def apply_physics(FINITE_ELEMENT, mesh_data):
    """
    Adiciona uma nova chave 'source' a cada dicionário em conn_data.
    
    Parâmetros:
    - mesh_data: Dicionário contendo os dados da malha.
    - element_type: Tuple (tipo do elemento, ordem).
    
    Retorna:
    - mesh_data: O dicionário atualizado com a chave 'source' em cada elemento de conn_data.
    """
    # Desempacotar o tipo de elemento
    type, order = FINITE_ELEMENT

    # Dictionary with all nodes in the mesh
    cell_data = mesh_data['cell']
    nodes_data = mesh_data['nodes']

    for key, cell in cell_data.items():
        # Calcular o valor de 'a' de acordo com a física do problema
        mur = cell['material']['relative_magnetic_permeability']
        er = cell['material']['relative_electric_permittivity']

        # Adicionar a nova chave 'material' ao dicionário da célula
        cell['stiffness_a_value'] = np.eye(2)
        cell['mass_a_value'] = - K0 ** 2
        
        # Adicionar a nova chave 'source' ao dicionário da célula
        cell['source'] = {'type': 'free_source', 'value': 0}

        # Dictionary with boundary nodes
        pec_nodes = {idx: node for idx, node in enumerate(cell['conn'])
                        if nodes_data[node]['bc']['type'] == 'PEC'}  

        bgt_nodes = {idx: node for idx, node in enumerate(cell['conn'])
                         if nodes_data[node]['bc']['type'] == 'BGT'}
        
        # Verifica tipo de fronteira absorvente (ABC)
        if len(pec_nodes) > 1:
            cell_data[key]['abc'] = {'type': 'PEC', 'conn_idx': pec_nodes}
            
        elif len(bgt_nodes) > 1:
            cell_data[key]['abc'] = {'type': 'BGT', 'conn_idx': bgt_nodes}
            
        else:
            cell_data[key]['abc'] = {'type': None, 'conn_idx': None}

    # Adiciona os potenciais de Dirichlet
    for key, node in nodes_data.items():
        if node['bc']['type'] == 'Dirichlet':
            # Coordenadas cartesianas do nó
            x, _ = node['xg']  
                        
            # Atualizar o valor de Dirichlet para o nó
            node['bc']['value'] = -np.exp(-1j * K0 * x)
    
    return mesh_data

# `ez_at_node()`

In [7]:
def ez_at_node(node, series_terms=40):
    ez = 0    
    ra = RADII['a']
    
    # Converter coordenadas cartesianas para coordenadas polares
    x, y = node['xg']
    rho, phi = np.sqrt(x**2 + y**2), np.arctan2(y, x)
    for n in range(0, series_terms):
        e_n = 2 if n != 0 else 1
        Jn_ka = jv(n, K0 * ra)
        Hn2_ka = hankel2(n, K0 * ra)
        Hn2_kp = hankel2(n, K0 * rho)
        ez += (-1j) ** n * e_n * (Jn_ka / Hn2_ka) * Hn2_kp * np.cos(n * phi)
    return ez

# `ez_at_omega()`

In [8]:
def ez_at_omega(mesh_data):
    ez = {}
    for key, node in mesh_data['nodes'].items():
        ez[key] = ez_at_node(node)
    return ez

# `ezh_at_contour()`

In [9]:
def ez_at_contour(mesh_data, contour='BGT'):
    ez = {}
    for key, node in mesh_data['nodes'].items():
        if node['bc']['type'] == contour:
            # Coordenadas cartesianas do nó
            x, y = node['xg']
            
            # Converter coordenadas cartesianas para coordenadas polares
            rho, phi = np.sqrt(x**2 + y**2), np.arctan2(y, x)

            # Converter phi para o intervalo [0, 2*pi]
            if phi < 0:
                phi = 2 * np.pi + phi

            # Analytical solution
            ez[key] = {
                'phi': phi,
                'ez': ez_at_node(node)
            }
    return ez

# `analytical_scattering()`

In [10]:
SCATTERER = {'name': 'circular_cylinder',
    'radii': {
        'TMz': [0.5],
        'TEz': []
    },
    'wavelength': 1,
    'amplitude': 1,
    'observation_angle': np.linspace(0, 2 * np.pi, 300),
    'iterations': 40
    }

class ObliqueIncidencePlaneWaveScattering():
    """
    This class represents a oblique incidence plane wave scattering from a circular cylinder 
    scatterer with a specific wavelength and amplitude.
    """

    def __init__(self, scatterer, polarization_mode, incident_angle):
        self.scatterer = scatterer
        self.wavelength = scatterer['wavelength']
        self.amplitude = scatterer['amplitude']
        self.phi = scatterer['observation_angle']
        self.radii = scatterer['radii'][polarization_mode]
        self.beta = 2 * np.pi / self.wavelength
        self.omega = 2 * np.pi * speed_of_light / self.wavelength
        self.theta_i = incident_angle
        self.mode = polarization_mode

    def compute_coefficient(self, n, beta_a):
        """
        This class computes the coefficient for the scattering width calculation.

        Args:
            n (_type_): _description_
            Ba (_type_): _description_

        Returns:
            _type_: _description_
        """
        if self.mode == 'TMz':
            # an coefficient
            return - jvp(n, beta_a * np.sin(self.theta_i), 0) / hankel2(
                n, beta_a * np.sin(self.theta_i))

        if self.mode == 'TEz':
            # bn coefficient
            return - jvp(n, beta_a * np.sin(self.theta_i), 1) / h2vp(
                n, beta_a * np.sin(self.theta_i), 1)

        return None  # Default return statement

    def scattering_width(self, radii_list, phi_list):

        # radii_list = self.scatterer['radii'][self.mode]
        # phi_list = self.scatterer['observation_angle']
        n_list = self.scatterer['iterations']

        sw_list = np.zeros((len(radii_list), len(phi_list)))
        for i, radius in enumerate(radii_list):
            beta_a = self.beta * radius
            for j, phi in enumerate(phi_list):
                summation = 0
                for n in range(n_list+1):
                    if n == 0:
                        en = 1
                    else:
                        en = 2
                    xn = self.compute_coefficient(n, beta_a)
                    summation += en * xn * np.cos(n * phi)
                sw_list[i, j] = 2 * self.wavelength / np.pi / \
                    np.sin(self.theta_i) * np.abs(summation) ** 2
        return sw_list
    
    def ez_scattered_field(self, rho, phi_offset):
        """
        Calculates the far zone scattered field for a given scatterer and phi offset.

        Parameters:
        scatterer (dict): A dictionary containing information about the scatterer.
        phi_offset (float): The offset angle in radians.

        Returns:
        np.ndarray: An array representing the far zone scattered field.
        """
        ez = np.zeros((len(self.radii), len(self.phi)), dtype='complex128')
        N = self.scatterer['iterations']

        for p, a in enumerate(self.radii):
            for q, phi in enumerate(self.phi):
                summ = 0
                for n in range(0, N):
                    e_n = 2 if n != 0 else 1
                    Jn_ka = jv(n, self.beta * a)
                    Hn2_ka = hankel2(n, self.beta * a)
                    Hn2_kp = hankel2(n, self.beta * rho)
                    summ += (-1j) ** n * e_n * (Jn_ka / Hn2_ka) * Hn2_kp * np.cos(n * (phi + phi_offset))
                ez[p, q] = summ            

        return ez
    
    def far_zone_scattered_field(self, phi_offset):
        """
        Calculates the far zone scattered field for a given scatterer and phi offset.

        Parameters:
        scatterer (dict): A dictionary containing information about the scatterer.
        phi_offset (float): The offset angle in radians.

        Returns:
        np.ndarray: An array representing the far zone scattered field.
        """
        ff_ez = np.zeros((len(self.radii), len(self.phi)), dtype='float64')
        N = self.scatterer['iterations']

        for i, a in enumerate(self.radii):
            for j, phi in enumerate(self.phi):
                summ = 0
                constant_term = np.sqrt(2 / np.pi / self.beta)
                for n in range(0, N):
                    e_n = 2 if n != 0 else 1
                    summ += e_n * np.cos(n * (phi + phi_offset)) * (
                        jv(n, self.beta * a) / hankel2(n, self.beta * a))
                ff_ez[i, j] = constant_term * np.abs(summ)
        return ff_ez

class Plotters:
    """This class contains methods for plotting the scattering results.

    """

    def __init__(self, scatterer, axis_flag):
        self.scatterer = scatterer
        self.wavelength = scatterer['wavelength']
        self.phi = [phi if phi >= 0 else phi + 2 * np.pi for phi in scatterer['observation_angle']]
        self.radii_list = scatterer['radii']
        self.flag = axis_flag

    def ez_field(self, ez, R):
        ez = ez.flatten()
        fig, axs = plt.subplots(2, 1, figsize=(8, 10), sharex=True)

        # Parte real
        axs[0].plot(np.degrees(self.phi), ez.real, label='Analytical', color='red')
        axs[0].set_ylabel(r'$R_e (E_{z}^{s})$')
        axs[0].set_title(f'Scattered Electric Field z-component\n'
                        rf'from a circular conducting cylinder ($R = {R}\lambda$)')
        axs[0].legend()
        axs[0].grid(False)

        # Parte imaginária
        axs[1].plot(np.degrees(self.phi), ez.imag, label='Analytical', color='blue')
        axs[1].set_xlabel(r'$\phi$ (degrees)')
        axs[1].set_ylabel(r'$I_m (E_{z}^{s})$')
        axs[1].legend()
        axs[1].grid(False)

        # Configurações compartilhadas
        for ax in axs:
            ax.set_xlim(0, 180)
            ax.set_xticks(np.arange(0, 181, 30))

        plt.tight_layout()
        plt.show()


    def ez_field_norm(self, ez, R):
        plt.figure(figsize=(8, 6))

        # Analytical formulation
        plt.plot(np.degrees(self.phi), np.abs(ez).flatten(),
                     label='Analytical', color='red')

        # Graphs adjustments
        plt.xlabel(r'$\phi$ (degrees)')
        plt.ylabel(r'$|E_{z}^{s}|$')
        plt.title(f'Scattered Electric Field z-component\n'
                  rf'from a circular conducting cylinder ($R = {R}\lambda$)')
        plt.xlim(0, 180)
        plt.grid(False)
        plt.xticks(np.arange(0, 181, 30))
        plt.legend()
        plt.tight_layout()
        plt.show()


    def scattering_width(self, sw_list, polarization_mode):
        """This method plots the scattering width of a circular cylinder.

        Args:
            sw_list (_type_): _description_
            radii_list (_type_): _description_
            mode (_type_): _description_
        """
        radii_list = self.scatterer['radii'][polarization_mode]

        plt.figure(figsize=(8, 6))
        for sw, radius in zip(sw_list, radii_list):
            plt.plot(np.degrees(self.phi), sw/self.wavelength,
                         label=f'a = ${radius}\\lambda$')
        plt.xlabel(r'$\phi$ (degrees)')
        plt.ylabel(r'SW ($\sigma_{2-D}/\lambda$)')
        plt.title(
            f'Two-dimensional {
                polarization_mode} bistatic scattering width (SW) '
            f'\n of a circular conducting cylinder'
        )
        plt.grid(False)
        plt.xlim(0, 180)
        # if polarization_mode == 'TMz' and self.flag:
        #     plt.ylim(0.2, 15)
        #     plt.yticks([0.2, 0.3, 0.5, 1, 2, 3, 5, 10, 15], [
        #                '0.2', '0.3', '0.5', '1', '2', '3', '5', '10', '15'])
        # elif polarization_mode == 'TEz' and self.flag:
        #     plt.ylim(0.01, 3)
        #     plt.yticks([0.01, 0.1, 0.2, 0.3, 0.5, 1, 2, 3], [
        #                '0.01', '0.1', '0.2', '0.3', '0.5', '1', '2', '3'])
        plt.legend()
        plt.tight_layout()
        plt.show()


    def far_field_norm(self, ff_ez):
        """
        Plots the surface induced current.

        Args:
            js (ndarray): The surface induced current calculated analytically.
            js_po (ndarray): The surface induced current calculated using the 
            Physical Optics formulation.
            js_mom (ndarray): The surface induced current calculated using the 
            Method of Moments.

        Returns:
            None
        """
        plt.figure(figsize=(8, 6))

        # Analytical formulation
        plt.semilogy(np.degrees(self.phi), ff_ez.flatten(),
                     label='Analytical', color='red')

        # # MoM formulation
        # plt.semilogy(np.degrees(self.phi), ff_ez_mom,
        #              label='MoM (CFIE-Point Matching)',
        #              color='black', linestyle='dotted')

        # # Physical Optics formulation
        # plt.semilogy(np.degrees(self.phi), ff_ez_po,
        #              label='Physical Optics (PO)',
        #              color='blue', linestyle='dashed')

        # Graphs adjustments
        plt.xlabel(r'$\phi$ (degrees)')
        plt.ylabel(r'$|E_{z}^{s}|/|E_{z}^{i}|$')
        plt.title(f'Far-Zone Scattered Electric Field z-component\n'
                  fr'from a circular conducting cylinder ($a = {self.radii_list['TMz'][0]}\lambda$)')
        plt.xlim(0, 180)
        plt.grid(False)
        plt.xticks(np.arange(0, 181, 30))
        plt.legend()
        plt.tight_layout()
        plt.show()

# 11.5 Plane Wave Scattering by Conducting Circular Cylinder
## 11.5.1 Normal Incidence Plane Wave Scattering by Conducting Circular Cylinder
tmz_mode = ObliqueIncidencePlaneWaveScattering(
    SCATTERER, polarization_mode='TMz', incident_angle=np.pi/2)

# Figure 11-13 Two-dimensional TMz bistatic scattering width (SW)
# of a circular conducting cylinder [1]
radii_list = SCATTERER['radii']['TMz']
phi = SCATTERER['observation_angle']
sw_tmz = tmz_mode.scattering_width(radii_list, phi)

# Scattered Electric Field z-component
R = 0.75
ez = tmz_mode.ez_scattered_field(R, phi_offset=0)

# Far-Zone Scattered Electric Field z-component
ff_ez = tmz_mode.far_zone_scattered_field(phi_offset=0)

# Plotting
# plotter = Plotters(SCATTERER, axis_flag=1)
# plotter.ez_field(ez, R)
# plotter.ez_field_norm(ez, R)
# plotter.far_field_norm(ff_ez)
# plotter.scattering_width(sw_tmz, 'TMz')

# `calculate_error()`

In [11]:
def calculate_error(u_exact, u_approx):
    """
    Calcula o erro entre a solução exata e a solução aproximada.

    Parâmetros:
    - u_exact: Array com os valores da solução exata u_ex.
    - u_approx: Array com os valores da solução aproximada u_h.

    Retorna:
    - erro: Valor do erro calculado.
    """
    # Verificar se os arrays têm o mesmo tamanho
    if len(u_exact) != len(u_approx):
        raise ValueError("Os arrays u_exact e u_approx devem ter o mesmo tamanho.")

    # Calcular a soma dos quadrados das diferenças
    N = len(u_exact)
    error = np.sqrt(np.sum((u_exact - u_approx)**2) / N)
    return np.abs(error)

# `plot_coordinates()`

In [12]:
def plot_coordinates(mesh_data):
    pec_coords = []  # Coordenadas de nós com tipo PEC
    bgt_coords = []  # Coordenadas de nós com tipo BGT

    for node in mesh_data['nodes'].values():
        x, y = node['xg']
        if node['bc']['type'] == 'Dirichlet':
            pec_coords.append((x, y))
        elif node['bc']['type'] == 'BGT':
            bgt_coords.append((x, y))

    # Conversão para listas separadas de x e y
    pec_x, pec_y = zip(*pec_coords) if pec_coords else ([], [])
    bgt_x, bgt_y = zip(*bgt_coords) if bgt_coords else ([], [])

    # Visualização
    plt.figure(figsize=(8, 8))
    plt.scatter(pec_x, pec_y, color='blue', label='PEC', marker='o', s=10)
    plt.scatter(bgt_x, bgt_y, color='red', label='BGT', marker='x', s=10)
    plt.axhline(0, color='gray', linewidth=0.5, linestyle='--')
    plt.axvline(0, color='gray', linewidth=0.5, linestyle='--')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend()
    plt.title("Visualização de Nós por Tipo de Condição de Contorno")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()

# `plot_ez_domain()`

In [13]:
def plot_ez_domain(FINITE_ELEMENT, mesh_data, u_dict, type='abs'):
    # Conversão do dicionário de soluções para lista
    if type == 'real':
        u = np.real(list(u_dict.values()))
    elif type == 'imag':
        u = np.imag(list(u_dict.values()))
    elif type == 'abs':
        u = np.abs(list(u_dict.values()))
    else:
        raise ValueError("Tipo de solução inválido. Use 'real', 'imag' ou 'abs'.")

    # Extração de informações sobre o elemento finito
    type, order = FINITE_ELEMENT

    # Extraindo as coordenadas globais dos nós (x, y) e a matriz de conectividade
    xg, yg, conn = graph_results.structured_data(mesh_data)

    # Verificação do tipo e ordem do elemento
    if type == "Triangle":
        triangulation = Triangulation(xg, yg, conn)
    else:
        raise ValueError("Apenas elementos triangulares ('Triangle') são suportados.")

    # Plot da solução
    plt.figure(figsize=(8, 6))
    plt.tricontourf(triangulation, u, cmap='viridis')
    plt.colorbar(label=r'$u(x, y)$')
    plt.triplot(triangulation, color='gray', alpha=0.5)
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.tight_layout()

    # Salvando o gráfico
    filepath = graph_results.get_dir(f"pos_processing/pictures/analytical_solution.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

# `apply_bgt_constants()`

In [14]:
def apply_bgt_constants(radius):
    # Radius domain
    R = RADII[radius]

    # Constantes de onda plana
    jk0 = 1j * K0

    BGT = {'0': {'ALPHA': jk0,
                  'BETA': 0},
            '1': {'ALPHA': jk0 + 1/(2*R),
                   'BETA': 0},
            '2': {'ALPHA': jk0 + 1/(2*R) - 1 / (8*R * (1 + jk0*R)),
                   'BETA': - 1 / (2*R * (1 + jk0*R))}}
    
    return BGT

# `plot_ez_profile()`

In [15]:
def plot_ez_profile(ez, ez_h):
    phi = [value['phi'] for value in ez.values()]
    ez = [value['ez'] for value in ez.values()]
    ez_h = [-value for value in ez_h.values()]

    # Parte Real de Ez
    plt.figure(figsize=(8, 6))
    plt.plot(np.degrees(phi), np.real(ez), 
            label='Analytical', color='blue')
    plt.plot(np.degrees(phi), np.real(ez_h), 
            label='FEM (BGT-2)', color='red', linestyle='--')
    plt.title(r"Scatterer Electric Field, $E_z$, at domain boundary $R = 1.5 \lambda$")
    plt.xlabel(r"$\phi$ $[deg]$")
    plt.ylabel(r"$Re(E_z)$ $[V/m]$")
    plt.legend()
    plt.xticks(np.arange(0, 361, 60))
    plt.grid(True)
    plt.xlim(0, 360)
    plt.show()

    # Parte Imaginária de Ez
    plt.figure(figsize=(8, 6))
    plt.plot(np.degrees(phi), np.imag(ez), 
            label='Analytical', color='blue')
    plt.plot(np.degrees(phi), np.imag(ez_h), 
            label='FEM (BGT-2)', color='red', linestyle='--')
    plt.title(r"Scatterer Electric Field, $E_z$, at domain boundary $R = 1.5 \lambda$")
    plt.xlabel(r"$\phi$ $[deg]$")
    plt.ylabel(r"$Im(E_z)$ $[V/m]$")
    plt.legend()
    plt.xticks(np.arange(0, 361, 60))
    plt.grid(True)
    plt.xlim(0, 360)
    plt.show()

    # Módulo de Ez
    plt.figure(figsize=(8, 6))
    plt.plot(np.degrees(phi), np.abs(ez), 
            label='Analytical', color='blue')
    plt.plot(np.degrees(phi), np.abs(ez_h), 
            label='FEM (BGT-2)', color='red', linestyle='--')
    plt.title(r"Scatterer Electric Field, $E_z$, at domain boundary $R = 1.5 \lambda$")
    plt.xlabel(r"$\phi$ $[deg]$")
    plt.ylabel(r"$|E_z|$ $[V/m]$")
    plt.legend()
    plt.xticks(np.arange(0, 361, 60))
    plt.grid(True)
    plt.xlim(0, 360)
    plt.show()

# `plot_ez_multi_profile()`

In [16]:
def plot_ez_multi_profile(ez_results, r_domain):
    phi = [value['phi'] for value in ez_results['ez'].values()]
    ez = [value['ez'] for value in ez_results['ez'].values()]
    
    # Parte Real de Ez
    plt.figure(figsize=(8, 6))
    # Resultado Numérico
    plt.plot(np.degrees(phi), np.real(ez), 
            label='Analytical', color='black')
    # Plotar os resultados para as diferentes condições de contorno
    for bgt in ['0', '1', '2']:
        ezh_bgt = [-value for value in ez_results[bgt].values()]
        plt.plot(np.degrees(phi), np.real(ezh_bgt), 
                label=f'BGT-{bgt}', linestyle='--')
    plt.title(fr"Scatterer Electric Field, $E_z$, at domain boundary $R = {r_domain}\lambda$")
    plt.xlabel(r"$\phi$ $[deg]$")
    plt.ylabel(r"$Re(E_z)$ $[V/m]$")
    plt.xticks(np.arange(0, 361, 60))
    plt.grid(True)
    plt.xlim(0, 360)    
    plt.legend()
    plt.show()

    # Parte Imaginária de Ez
    plt.figure(figsize=(8, 6))
    # Resultado Numérico
    plt.plot(np.degrees(phi), np.imag(ez), 
            label='Analytical', color='black')
    # Plotar os resultados para as diferentes condições de contorno
    for bgt in ['0', '1', '2']:
        ezh_bgt = [-value for value in ez_results[bgt].values()]
        plt.plot(np.degrees(phi), np.imag(ezh_bgt), 
                label=f'BGT-{bgt}', linestyle='--')
    plt.title(fr"Scatterer Electric Field, $E_z$, at domain boundary $R = {r_domain}\lambda$")
    plt.xlabel(r"$\phi$ $[deg]$")
    plt.ylabel(r"$Im(E_z)$ $[V/m]$")
    plt.xticks(np.arange(0, 361, 60))
    plt.grid(True)
    plt.xlim(0, 360)    
    plt.legend()
    plt.show()

    # Módulo de Ez
    plt.figure(figsize=(8, 6))
    # Resultado Numérico
    plt.plot(np.degrees(phi), np.abs(ez), 
            label='Analytical', color='black')
    # Plotar os resultados para as diferentes condições de contorno
    for bgt in ['0', '1', '2']:
        ezh_bgt = [-value for value in ez_results[bgt].values()]
        plt.plot(np.degrees(phi), np.abs(ezh_bgt), 
                label=f'BGT-{bgt}', linestyle='--')
    plt.title(fr"Scatterer Electric Field, $E_z$, at domain boundary $R = {r_domain}\lambda$")
    plt.xlabel(r"$\phi$ $[deg]$")
    plt.ylabel(r"$|E_z|$ $[V/m]$")
    plt.xticks(np.arange(0, 361, 60))
    plt.grid(True)
    plt.xlim(0, 360)    
    plt.legend()
    plt.show()

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.